In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
 !kill -9 -1 #(перезагрузка collab)

In [ ]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [ ]:
import torchvision.models as models
import torch.nn as nn

model = models.resnet18(pretrained=True)
num_classes = 20
model.fc = nn.Linear(in_features=512, out_features=num_classes)
nn.init.xavier_uniform_(model.fc.weight)
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True
model

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!ls -la /gdrive/My\ Drive/ML
!cp /gdrive/My\ Drive/ML/VOC2007.tar.gz .
!ls .
!tar -xvf VOC2007.tar.gz

In [ ]:
from  torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import numpy as np
IMAGE_FOLDER_ROOT = 'sample_data/VOC2007/VOC2007'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
data_train = ImageFolder(IMAGE_FOLDER_ROOT+'/trainval',transform=transforms.Compose([transforms.Resize(224),transforms.RandomCrop(224),transforms.ToTensor(),normalize]))
data_test = ImageFolder(IMAGE_FOLDER_ROOT+'/test',transform=transforms.Compose([transforms.Resize(224),transforms.RandomCrop(224),transforms.ToTensor(),normalize]))

In [ ]:
!pip install Pillow==4.1.1
!pip install PIL
!pip install image
import PIL.Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
train_loader = torch.utils.data.DataLoader(data_train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=64, shuffle=True)
print(len(data_train), len(data_test))

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
import torch.optim
import matplotlib.pyplot as plt
import numpy as np

def accuracy(output,labels):
  predictions = torch.argmax(output,dim=1)
  correct = (predictions == labels).sum().cpu().numpy()
  return correct / len(labels)

print(device)

In [ ]:
model.eval()

accuracies = []
with torch.no_grad():
  for itr,x in enumerate(test_loader):
    data = x[0]
    labels = x[1]

    data = data.to(device)
    labels = labels.to(device)

    y_pred = model.forward(data)
    acc = accuracy(y_pred,labels)
    accuracies.append(acc)


print('accuracy {:.2f}'.format(np.mean(accuracies)))

In [ ]:
from torch.optim import lr_scheduler
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

lossf = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.95, nesterov=True)

# Decay LR by a factor of 0.1 every 7 epochs
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [ ]:

model.to(device)
model.train()


for epoch in range(6):
  lr_scheduler.step()
  for itr,x in enumerate(train_loader):
    data = x[0]
    labels = x[1]
    
    optimizer.zero_grad()

    data = data.to(device)
    labels = labels.to(device)
    
    y_pred = model.forward(data)
    
    loss = lossf(y_pred,labels)
    loss.backward()
    optimizer.step()

    print('Iteration {}, train accuracy {:.2f}, loss {:.4f}'.format(itr+epoch*len(train_loader),accuracy(y_pred,labels),loss))

In [ ]:
model.eval()

accuracies = []
with torch.no_grad():
  for itr,x in enumerate(test_loader):
    data = x[0]
    labels = x[1]

    optimizer.zero_grad()

    data = data.to(device)
    labels = labels.to(device)

    y_pred = model.forward(data)
    accuracies.append(accuracy(y_pred,labels))
    

print('accuracy {:.2f}'.format(np.mean(accuracies)))
m_accuracy = np.mean(accuracies)

In [ ]:
torch.save({
            'accuracy': m_accuracy,
            'model_state_dict': model.state_dict(),
            'model': model,
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, "/gdrive/My Drive/ML/CLTresnet18")